In [53]:
import pandas as pd
import lingam
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.preprocessing import OrdinalEncoder, StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import math

import ylearn
from ylearn.causal_discovery import CausalDiscovery
model=lingam.DirectLiNGAM()

In [54]:
from lingam.utils import make_dot

In [55]:
train = pd.read_csv('train.csv')
for col in train.columns:
    if train[col].dtype == 'object':
        label_encoder = LabelEncoder().fit(train[col])
        train[col] = label_encoder.transform(train[col])

In [18]:
train_na_dic = {}

for col in train.columns:
    train_na_dic[col] = train[col].isna().sum()/train.shape[0]
    
na_fill_need_col = [x[0] for x in train_na_dic.items() if x[1]>0]

from xgboost.sklearn import XGBClassifier,XGBRegressor
training_col = [x for x in train.columns if x not in ['treatment','outcome']]

for col in na_fill_need_col:
    train_cols = [x for x in training_col if x != col and 'predict' not in x]
    train_na_df = train[~train[col].isna()]
    xgb = XGBRegressor()
    xgb.fit(train_na_df[train_cols],train_na_df[col])
    train[col + '_predict'] = xgb.predict(train[train_cols])
    #test[col + '_predict'] = xgb.predict(test[train_cols])

In [19]:
def fillna(x1,x2):
    if math.isnan(x1):
        return x2
    else:
        return x1
    
for col in na_fill_need_col:
    train[col] = train.apply(lambda x:fillna(x[col],x[col+'_predict']),axis=1) 
    #test[col] = test.apply(lambda x:fillna(x[col],x[col+'_predict']),axis=1) 

In [20]:
cols = ['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'V_11', 'V_12', 'V_13', 'V_14', 'V_15', 'V_16', 'V_17', 'V_18',
       'V_19', 'V_20', 'V_21', 'V_22', 'V_23', 'V_24', 'V_25', 'V_26', 'V_27',
       'V_28', 'V_29', 'V_30', 'V_31', 'V_32', 'V_33', 'V_34', 'V_35', 'V_36',
       'V_37', 'V_38', 'V_39', 'treatment', 'outcome']

In [23]:
train[cols].to_csv("train_done.csv",index=False)

In [32]:
train.tail()

,V_0,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,...,V_34,V_35,V_36,V_37,V_38,V_39,treatment,outcome,V_4_predict,V_28_predict
36183,0.826893,4.331358,0.0,-1.551377,-2.904315,0.0,-46.2,1.250,0,999,...,7.119088,5.488357,33,-1.8,1.0,0.0,2,20.512318,-2.799276,177.006546
36184,0.631707,1.383925,0.0,-0.411007,4.483282,1.0,-42.7,4.962,0,999,...,-0.408196,1.127706,42,1.4,0.0,4.0,0,-3.178066,5.066493,333.978729
36185,-0.072426,4.918745,0.0,-0.351575,-4.926085,1.0,-36.1,4.963,0,999,...,-7.168536,3.873154,46,1.4,0.0,2.0,0,-43.733190,-4.732676,138.499115
36186,-1.029582,4.633457,0.0,-0.496135,3.944163,1.0,-46.2,1.334,0,999,...,-6.115295,7.455648,39,-1.8,1.0,4.0,2,12.905083,3.446393,160.316284
36187,0.812233,3.682767,1.0,0.999073,3.450035,1.0,-36.4,4.859,0,999,...,-4.069095,-1.137845,39,1.1,1.0,0.0,1,-31.867275,2.841186,349.751465


In [24]:
model=lingam.DirectLiNGAM()
model.fit(train[cols])

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\lingam\direct_lingam.py:221: RuntimeWarning: invalid value encountered in true_divide
  xj_std = (X[:, j] - np.mean(X[:, j])) / np.std(X[:, j])
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\lingam\direct_lingam.py:220: RuntimeWarning: invalid value encountered in true_divide
  xi_std = (X[:, i] - np.mean(X[:, i])) / np.std(X[:, i])
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\lingam\direct_lingam.py:148: RuntimeWarning: invalid value encountered in double_scalars
  return xi - (np.cov(xi, xj)[0, 1] / np.var(xj)) * xj
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLar

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

In [25]:
d = {}
for i in range(39):
    d[cols[i]] = model.estimate_total_effect(train[cols],i,41)

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

In [62]:
model.estimate_total_effect??

In [26]:
d2 = {}
for i in range(39):
    d2[cols[i]] = model.estimate_total_effect(train[cols],i,40)

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in

In [67]:
model.estimate_total_effect(train[cols],9,40)

C:\Users\Yazhou Fu\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


0.0

In [68]:
train[cols]

,V_0,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,...,V_32,V_33,V_34,V_35,V_36,V_37,V_38,V_39,treatment,outcome
0,1.778505,2.376919,0.0,-1.680176,-1.581142,0.0,-31.4,0.838,0,3,...,36.710780,5.438752,-1.269324,6.026618,53,-2.9,0.0,3.0,2,4.745129
1,-0.631728,5.337631,0.0,-0.869348,-2.767838,0.0,-42.0,4.120,0,999,...,32.913967,3.478513,3.309444,3.011011,50,-0.1,2.0,4.0,0,7.310963
2,-0.862595,3.215953,1.0,-1.614799,-2.614270,1.0,-41.8,4.866,0,999,...,-201.735840,1.514067,-5.461385,4.281093,37,1.4,1.0,2.0,2,-52.156193
3,0.230854,2.250193,0.0,-1.631994,-5.705525,1.0,-36.1,4.966,0,999,...,-69.010270,3.073917,3.285422,3.228095,35,1.4,0.0,3.0,0,-17.011566
4,0.194774,4.896442,0.0,-0.143322,2.568728,1.0,-42.7,4.968,0,999,...,-46.994690,2.903061,-1.614107,4.383740,58,1.4,0.0,2.0,2,-15.229788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36183,0.826893,4.331358,0.0,-1.551377,-2.904315,0.0,-46.2,1.250,0,999,...,102.783640,3.302112,7.119088,5.488357,33,-1.8,1.0,0.0,2,20.512318
36184,0.631707,1.383925,0.0,-0.411007,4.483282,1.0,-42.7,4.962,0,999,...,-11.040594,2.109287,-0.408196,1.127706,42,1.4,0.0,4.0,0,-3.178066
36185,-0.072426,4.918745,0.0,-0.351575,-4.926085,1.0,-36.1,4.963,0,999,...,-180.988000,3.585552,-7.168536,3.873154,46,1.4,0.0,2.0,0,-43.733190
36186,-1.029582,4.633457,0.0,-0.496135,3.944163,1.0,-46.2,1.334,0,999,...,70.905310,3.965726,-6.115295,7.455648,39,-1.8,1.0,4.0,2,12.905083


In [27]:
cols = ['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'V_11', 'V_12', 'V_13', 'V_14', 'V_15', 'V_16', 'V_17', 'V_18',
       'V_19', 'V_20', 'V_21', 'V_22', 'V_23', 'V_24', 'V_25', 'V_26', 'V_27',
       'V_28', 'V_29', 'V_30', 'V_31', 'V_32', 'V_33', 'V_34', 'V_35', 'V_36',
       'V_37', 'V_38', 'V_39', 'treatment', 'outcome']

edges = []
for i,index in enumerate(cols):
    adj_matrix = model.adjacency_matrix_[i]
    for j,val in enumerate(adj_matrix):
        if val > 0.01:
            edges.append((cols[i],cols[j],val))

In [40]:
edges

[('V_6', 'V_2', 2.0989962313177672),
 ('V_7', 'V_2', 1.1064961424611732),
 ('V_7', 'V_3', 0.01143442208381248),
 ('V_7', 'V_6', 0.07355875181333206),
 ('V_9', 'V_2', 4.858127686205644),
 ('V_9', 'V_7', 26.970301610896453),
 ('V_12', 'V_5', 0.3511162120997324),
 ('V_12', 'V_6', 0.04454361081337508),
 ('V_12', 'V_8', 0.1692854073843552),
 ('V_14', 'V_10', 0.04463460423553753),
 ('V_16', 'V_2', 0.01128045259925599),
 ('V_16', 'V_7', 0.018758804500195864),
 ('V_16', 'V_8', 0.034439064029154025),
 ('V_19', 'V_6', 0.018342296237628244),
 ('V_19', 'V_12', 0.190655407335439),
 ('V_21', 'V_2', 0.16307791687975728),
 ('V_21', 'V_7', 0.19887254885009342),
 ('V_21', 'V_12', 0.011952709921125473),
 ('V_21', 'V_16', 0.07863219204639946),
 ('V_22', 'V_0', 2.909791093877685),
 ('V_22', 'V_1', 3.397128121592289),
 ('V_22', 'V_2', 0.1895792902228356),
 ('V_22', 'V_4', 3.4572712914461285),
 ('V_22', 'V_5', 4.5066596144345965),
 ('V_22', 'V_6', 0.09584576527091059),
 ('V_22', 'V_7', 2.0155612726379486),
 

In [28]:
#['V_2','V_15', 'V_28',  'V_34','V_33','V_31']

In [74]:
d2

{'V_0': 0.0,
 'V_1': 0.13309036018584472,
 'V_2': 0.7196814464146286,
 'V_3': 0.03249228568232572,
 'V_4': 0.0,
 'V_5': -0.025882555856880932,
 'V_6': -0.004722685899881068,
 'V_7': 0.030393312734577817,
 'V_8': 0.0,
 'V_9': 0.0,
 'V_10': 0.0,
 'V_11': -0.03414019930135726,
 'V_12': 0.0,
 'V_13': 0.01745946581167929,
 'V_14': 0.0,
 'V_15': 0.0,
 'V_16': 0.0,
 'V_17': 0.0,
 'V_18': -0.09756635227324782,
 'V_19': -0.003228968090079927,
 'V_20': 0.0,
 'V_21': 0.0,
 'V_22': 0.0008547251476367985,
 'V_23': 0.011415162628939365,
 'V_24': 0.16996359809424358,
 'V_25': 0.0,
 'V_26': 0.0,
 'V_27': 0.0,
 'V_28': 0.0,
 'V_29': 0.019575866310069977,
 'V_30': -0.0031449209786614377,
 'V_31': 0.0,
 'V_32': 0.005555500314255275,
 'V_33': 0.09267864886594462,
 'V_34': 0.013559889378524046,
 'V_35': 0.11298668045609511,
 'V_36': -0.0020335305276460913,
 'V_37': -0.03733560191311232,
 'V_38': 0.0}

In [72]:
for key in sorted(d.items(),key = lambda x:x[1],reverse = True):
    print(key[0],d[key[0]],d2[key[0]])

V_15 9.542813764543213 0.0
V_8 3.796138125698013 0.0
V_1 0.9297346267691936 0.13309036018584472
V_19 0.7492984223879232 -0.003228968090079927
V_33 0.4248100332457081 0.09267864886594462
V_31 0.3886557821934432 0.0
V_24 0.31450964812068877 0.16996359809424358
V_12 0.21337744807172848 0.0
V_13 0.19655855578392206 0.01745946581167929
V_32 0.1482630551977924 0.005555500314255275
V_29 0.1190403705992407 0.019575866310069977
V_37 0.08963968172652095 -0.03733560191311232
V_23 0.08479546903483588 0.011415162628939365
V_34 0.083524615406759 0.013559889378524046
V_35 0.024634070980556904 0.11298668045609511
V_36 0.014891514612790122 -0.0020335305276460913
V_22 0.007798757481383468 0.0008547251476367985
V_0 0.0 0.0
V_3 0.0 0.03249228568232572
V_4 0.0 0.0
V_11 0.0 -0.03414019930135726
V_17 0.0 0.0
V_20 0.0 0.0
V_26 0.0 0.0
V_27 0.0 0.0
V_38 0.0 0.0
V_28 -0.007434517407463293 0.0
V_9 -0.020659589245458642 0.0
V_21 -0.0643822112023309 0.0
V_25 -0.14779986699304137 0.0
V_6 -0.17692599699937053 -0.004

In [30]:
sorted(d2.items(),key = lambda x:x[1],reverse = True)

[('V_2', 0.7196814464146286),
 ('V_24', 0.16996359809424358),
 ('V_1', 0.13309036018584472),
 ('V_35', 0.11298668045609511),
 ('V_33', 0.09267864886594462),
 ('V_3', 0.03249228568232572),
 ('V_7', 0.030393312734577817),
 ('V_29', 0.019575866310069977),
 ('V_13', 0.01745946581167929),
 ('V_34', 0.013559889378524046),
 ('V_23', 0.011415162628939365),
 ('V_32', 0.005555500314255275),
 ('V_22', 0.0008547251476367985),
 ('V_0', 0.0),
 ('V_4', 0.0),
 ('V_8', 0.0),
 ('V_9', 0.0),
 ('V_10', 0.0),
 ('V_12', 0.0),
 ('V_14', 0.0),
 ('V_15', 0.0),
 ('V_16', 0.0),
 ('V_17', 0.0),
 ('V_20', 0.0),
 ('V_21', 0.0),
 ('V_25', 0.0),
 ('V_26', 0.0),
 ('V_27', 0.0),
 ('V_28', 0.0),
 ('V_31', 0.0),
 ('V_38', 0.0),
 ('V_36', -0.0020335305276460913),
 ('V_30', -0.0031449209786614377),
 ('V_19', -0.003228968090079927),
 ('V_6', -0.004722685899881068),
 ('V_5', -0.025882555856880932),
 ('V_11', -0.03414019930135726),
 ('V_37', -0.03733560191311232),
 ('V_18', -0.09756635227324782)]

In [31]:
for col in ['V_2', 'V_15', 'V_28',  'V_34','V_33','V_31']: #28
    print(d[col],d2[col])

-2.791754794349466 0.7196814464146286
9.542813764543213 0.0
-0.007434517407463293 0.0
0.083524615406759 0.013559889378524046
0.4248100332457081 0.09267864886594462
0.3886557821934432 0.0


In [ ]:
2 3 8 16

In [ ]:
# V_2 -> treatment [label=0.63]
# 	V_24 -> treatment [label=0.21]
# 	V_2 -> outcome [label=-0.59]
# 	V_3 -> outcome [label=0.69]
# 	V_8 -> outcome [label=0.56]
# 	V_10 -> outcome [label=-0.23]
# 	V_11 -> outcome [label=0.34]
# 	V_13 -> outcome [label=0.46]
# 	V_14 -> outcome [label=-0.39]
# 	V_15 -> outcome [label=0.25]
# 	V_16 -> outcome [label=-0.64]
# 	V_33 -> outcome [label=0.48]
# 	V_39 -> outcome [label=0.31] 

In [ ]:
# V_2 -> treatment [label=0.63]
# 	V_16 -> treatment [label=0.15]
# 	V_24 -> treatment [label=0.21]
# 	V_33 -> treatment [label=0.12]
# 	V_35 -> treatment [label=0.11]
    
    
# 	V_0 -> outcome [label=-0.14]
# 	V_2 -> outcome [label=-0.59]
# 	V_3 -> outcome [label=0.69]
# 	V_8 -> outcome [label=0.56]
# 	V_10 -> outcome [label=-0.23]
# 	V_11 -> outcome [label=0.34]
# 	V_13 -> outcome [label=0.46]
# 	V_14 -> outcome [label=-0.39]
# 	V_15 -> outcome [label=0.25]
# 	V_16 -> outcome [label=-0.64]
# 	V_18 -> outcome [label=0.11]
# 	V_31 -> outcome [label=0.14]
# 	V_32 -> outcome [label=0.15]
# 	V_33 -> outcome [label=0.48]
# 	V_39 -> outcome [label=0.31]

In [ ]:
# 	V_2 -> treatment [label=0.63]
# 	V_24 -> treatment [label=0.21]
# 	V_2 -> outcome [label=-0.59]
# 	V_3 -> outcome [label=0.69]
# 	V_8 -> outcome [label=0.56]
# 	V_10 -> outcome [label=-0.23]
# 	V_11 -> outcome [label=0.34]
# 	V_13 -> outcome [label=0.46]
# 	V_14 -> outcome [label=-0.39]
# 	V_15 -> outcome [label=0.25]
# 	V_16 -> outcome [label=-0.64]
# 	V_33 -> outcome [label=0.48]
# 	V_39 -> outcome [label=0.31]

In [1]:
['V_2', 'V_24', 'V_3',  'V_8', 'V_13', 'V_11',
    'V_14', 'V_15',  'V_16', 'V_33', 'V_39']

['V_2',
 'V_24',
 'V_3',
 'V_8',
 'V_13',
 'V_11',
 'V_14',
 'V_15',
 'V_16',
 'V_33',
 'V_39']

In [2]:
['V_2','V_15', 'V_28',  'V_34','V_33','V_31']

['V_2', 'V_15', 'V_28', 'V_34', 'V_33', 'V_31']

In [52]:
dot = make_dot(model.adjacency_matrix_, labels =cols)

In [54]:
# dot.render('dag')